In [1]:
import pandas as pd
from datetime import date, timedelta
import re
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# calcula tempo do workflow
def calculate_time_wf(inicio, fim):    
    delta = fim - inicio
    return int(delta.seconds/60)

# calcula idade
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

# cria dummies de periodo do dia
def set_shift(x):
    if 0<x<=6:
        return 'madrugada'
    elif 6<x<=11:
        return 'manha'
    elif 11<x<=14:
        return 'almoco'
    elif 14<x<=17:
        return 'tarde'
    elif 17<x<=19:
        return 'janta'
    else:
        return 'noite'

In [1]:
mes=['Jun','Jul','Ago','Set','Out','Nov','Dez','Jan','Fev', 'Mar','Abr', 'Jun2019', 'Jul2019']
#mes=['Fev']

#versao='v2.0'

listMultisat = []

for i,j in enumerate(mes):
    dfaux=pd.read_csv('/home/dev/poc/arquivos/model/in/Query_Multisat_'+str(j)+'.txt',sep='\t',header=0,
                    names=['nrviagem','placa','cidadeorigem','estadoorigem',
                           'cidadedestino','estadodestino','vlrtotal',
                           'tpoperacao','tpocorrencia','urbana','prioridade',
                           'dtiniciowf','dtfimwf','nomemotorista',
                           'cpfmotorista','tpcarga','tptecnologia',
                           'cto'],error_bad_lines=False,low_memory=False)
    listMultisat.append(dfaux)
    
df = pd.concat(listMultisat, axis = 0, ignore_index = True)

NameError: name 'pd' is not defined

In [5]:
# tratar tipos de dados
df['dtiniciowf'] = pd.to_datetime(df['dtiniciowf'])
df['dtfimwf'] = pd.to_datetime(df['dtfimwf'], errors='coerce').fillna(pd.to_datetime(df['dtiniciowf']) + timedelta(minutes=1))

In [6]:
df['cpfmotorista'] = pd.to_numeric(df['cpfmotorista'],errors='coerce')

In [7]:
# agrupar por viagem e totalizando por tecnologia
df1 = df.drop(['placa','cidadeorigem','estadoorigem','cidadedestino',
               'estadodestino','vlrtotal','tpoperacao','tpocorrencia',
               'urbana','prioridade','dtiniciowf','dtfimwf','nomemotorista',
               'cpfmotorista','tpcarga','cto'], axis=1)

tecnologias=['ONX','OMN','SAS','SGH','TRC','LOC','ATC','OUTRAS']

# valida tipos de tecnologias válidas - caso identifique alguma diferente classifica como OUTRAS
df1.loc[df1['tptecnologia'].isin(tecnologias)==False, 'tptecnologia'] = 'OUTRAS'

# transforma linhas em colunas por tipo de tecnologia
df1 = pd.get_dummies(df1, prefix=['t_'], columns=['tptecnologia'])

# criar colunas que estejam faltando
tecnologias = ['t__{0}'.format(t) for t in tecnologias]

for t in tecnologias:
    if t not in df1.columns:
        df[t] = 0

df1 = df1.groupby(['nrviagem'], as_index=False).sum()

In [8]:
# agrupar por viagem e totalizando por prioridade
df2 = df.drop(['placa','cidadeorigem','estadoorigem','cidadedestino',
               'estadodestino','vlrtotal','tpoperacao','tpocorrencia',
               'urbana','dtiniciowf','dtfimwf','nomemotorista',
               'cpfmotorista','tpcarga','tptecnologia','cto'], axis=1)

prioridades=[1,2,3,4,5,6,7,9,10,11,12,13,15,18,19,20,95,96,97,98,99]

# valida as prioridades válidas
df2.loc[df2['prioridade'].isin(prioridades)==False, 'prioridade'] = 0

# transforma linhas em colunas por prioridade
df2 = pd.get_dummies(df2, prefix=['p_'], columns=['prioridade'])

# remover colunas com prioridade = 0 (p__0)
if 'p__0' in df.columns:
    del df2['p__0']

# criar colunas que estejam faltando
prioridades = ['p__{0}'.format(p) for p in prioridades]

for p in prioridades:
    if p not in df.columns:
        df2[p] = 0

df2 = df2.groupby(['nrviagem'], as_index=False).sum()

In [9]:
# agrupar por viagem e tratar a ocorrencia de duplicidades de data de inicio e cpfs 
df3 = df.drop(['placa','cidadeorigem','estadoorigem','cidadedestino','estadodestino',
               'vlrtotal','tpoperacao','tpocorrencia','urbana','prioridade',
               'dtfimwf','nomemotorista','tpcarga',
               'tptecnologia'], axis=1)

df3 = df3.groupby(['nrviagem'], as_index=False).max()

In [10]:
df5 = pd.read_csv('/home/dev/poc/arquivos/model/in/Query_Motoristas.txt', sep='\t', 
                 names=['nomemotorista','endereco','nomepai',
                        'nomemae','dtnasc', 'rg', 'cpfmotorista',
                        'orgaoemissor','uf','dtexpedicaorg','sexo',
                        'estadocivil','nrprocessosinistro','possuisinistro',
                        'tiposinistro','dtunknow','ultimaempresa'],
                 error_bad_lines=False)

df5 = df5.drop(['nomemotorista','endereco','nomepai','nomemae',
                'rg','orgaoemissor','uf','dtexpedicaorg','sexo',
                'nrprocessosinistro','tiposinistro','dtunknow',
                'ultimaempresa'], axis=1)

# trata dados de motorista
df5['cpfmotorista'] = pd.to_numeric(df5['cpfmotorista'],errors='coerce')
df5['dtnasc'] = pd.to_datetime(df5['dtnasc'],errors='coerce')
df5['idade'] = df5['dtnasc'].apply(calculate_age)

# trata problema referente a dados repetidos
df5 = df5.groupby(['cpfmotorista'], as_index=False).max()
df5.drop_duplicates(keep='first', inplace=True)

In [11]:
# une dados de ocorrencia com dados de motorista pelo CPF
df5 = pd.merge(df3, df5, on='cpfmotorista', how='left')

del(df3)

# trata dados com NA
df5.fillna(0, inplace=True)

In [12]:
# cria variaveis adicionais
df['diasemanawf'] = df['dtiniciowf'].apply( lambda x: x.dayofweek) # 0 - Segunda até 6 - Domingo
df['nrdiasiniciomeswf'] = df['dtiniciowf'].apply( lambda x: x.day) # nr dias desde o inicio do mês
df['turno']=df['dtiniciowf'].dt.hour
df['turno']=df['turno'].apply(set_shift)


turnos=['almoco','janta','madrugada','manha','noite','tarde']

# transforma linhas em colunas por mercadoria
df = pd.get_dummies(df, prefix=['s'],prefix_sep=[''],columns=['turno'])

# criar colunas que estejam faltando
turnos = ['s{0}'.format(t) for t in turnos]

for t in turnos:
    if t not in df.columns:
        df[t] = 0

df = df.drop(['cpfmotorista','nomemotorista','tptecnologia','prioridade','cto'],axis=1)
df = pd.merge(df, df5, on=['nrviagem','dtiniciowf'], how='inner')
df = pd.merge(df, df1, on=['nrviagem'], how='inner')
df = pd.merge(df, df2, on=['nrviagem'], how='inner')

del(df5)
del(df1)
del(df2)

In [13]:
df['tempowf'] = df.apply(lambda row: calculate_time_wf(row['dtiniciowf'], row['dtfimwf']),axis=1)

In [14]:
#print(df.reset_index().sort_values(by=['tempowf'], ascending=False)[['nrviagem','tpocorrencia', 'tempowf','dtiniciowf', 'dtfimwf']])

In [15]:
# elimina registros duplicados
df.drop_duplicates(keep='first', inplace=True)

In [16]:
# Avaliando as cidades de destino
#cidades=df['cidadedestino'].value_counts()
#pct = (cidades/cidades.sum())*100
#cum=pct.cumsum()
#cidades=pd.concat([cidades,pct,cum],axis=1,keys=['total','perct.','cumulative'])

# seleciona aqui o percentual acumulado de interesse
#cidades=cidades[cidades['cumulative']<70].index.tolist()

In [17]:
# Avaliando as cidades de destino
#cidades=df['cidadedestino'].value_counts()
#pct = (cidades/cidades.sum())*100
#cum=pct.cumsum()
#cidades=pd.concat([cidades,pct,cum],axis=1,keys=['total','perct.','cumulative'])

# seleciona aqui o percentual acumulado de interesse
# cidades=cidades[cidades['cumulative']<70].index.tolist()

cidades=['ANAPOLIS','APARECIDA DE GOIANIA','ARAUCARIA','BARRA MANSA','BARUERI','BAURU','BELEM',
         'BELO HORIZONTE','BETIM','BLUMENAU','BRASILIA','CAMPINAS','CANOAS','CARAZINHO',
         'CAXIAS DO SUL','CONTAGEM','CUBATAO','CUIABA','CURITIBA','DUQUE DE CAXIAS','EMBU DAS ARTES',
         'FEIRA DE SANTANA','FORTALEZA','GASPAR','GOIANIA','GUARULHOS','HORTOLANDIA',
         'ITAJAI','JABOATAO DOS GUARARAPES','JOINVILLE','JUNDIAI','LONDRINA','LOUVEIRA',
         'MANAUS','NAVEGANTES','NOVA SANTA RITA','OSASCO','OUTRAS','PARANAGUA','PAULINIA','PORTO ALEGRE',
         'RIBEIRAO PRETO','RIO DE JANEIRO','SALVADOR','SAO BERNARDO DO CAMPO','SAO JOSE DOS PINHAIS',
         'SAO PAULO','SERRA','SIMOES FILHO','SUMARE','UBERLANDIA','VIANA','VILA VELHA',
         'VITORIA DE SANTO ANTAO']

# valida cidades válidas - caso identifique alguma diferente classifica como OUTRAS
df.loc[df['cidadedestino'].isin(cidades)==False, 'cidadedestino'] = 'OUTRAS'

# transforma linhas em colunas por cidade
df = pd.get_dummies(df, prefix=['c_'], columns=['cidadedestino'])

# criar colunas que estejam faltando
cidades = ['c__{0}'.format(c) for c in cidades]

for c in cidades:
    if c not in df.columns:
        df[c] = 0

In [18]:
# Avaliando as mercadorias transportadas
#cargas=df['tpcarga'].value_counts()
#pct= (cargas/cargas.sum())*100
#cum=pct.cumsum()
#cargas=pd.concat([cargas,pct,cum],axis=1,keys=['total','perct.','cumulative'])

# seleciona aqui o percentual acumulado de interesse
#cargas=cargas[cargas['cumulative']<99.7].index.tolist()

mercadorias=['ACUCAR','AUTO-PECAS','BEBIDAS','CALCADOS','CARGA FRACIONADA','CIGARROS','COMBUSTIVEL',
             'COSMETICOS','DEFENSIVO AGRICOLA','DIVERSOS','DIVERSOS PRODUTOS','EQUIPAMENTOS ELETRICOS',
             'ESPECIFICOS','ESTIRENO','FIOS E CABOS','MEDICAMENTOS','NESTLE','OUTRAS','PECAS AUTOMOTIVAS',
             'POLIESTIRENO','POLIETILENO','PRODUTO ACABADO','PRODUTOS QUIMICOS','TINTAS E SOLVENTES']

# classifica como OUTRAS as demais mercadorias que nao fiquem dentro do percentual de interesse
df.loc[df['tpcarga'].isin(mercadorias)==False, 'tpcarga'] = 'OUTRAS'

# converte as linhas para colunas por mercadorias
df = pd.get_dummies(df, prefix=['m_'], columns=['tpcarga'])

# criar colunas que estejam faltando
mercadorias = ['m__{0}'.format(m) for m in mercadorias]

for m in mercadorias:
    if m not in df.columns:
        df[m] = 0

In [19]:
# leitura do arquivo de sinistros (reguladora)
#mes=['Jan']
#mes=['Jun','Jul','Ago','Set','Out','Nov','Dez','Jan','Fev']

listSinistros = []

for i,j in enumerate(mes):
    dfaux = pd.read_csv('/home/dev/poc/arquivos/model/in/Query_Sinistros_'+str(j)+'.txt', sep='\t', header=0,
                 names=['cdprocesso', 'nrprocesso', 'mercadoria', 'nrviagem', 
                        'tpsinistro', 'tpoperacao', 'txtunknow', 
                        'vlrgasto','placa', 'modeloveiculo', 
                        'tpveiculo', 'idadeveiculo', 'coordenadas', 
                        'naturezasinistro', 'dtevento', 'hrevento'],                 
                 error_bad_lines=False)    
    listSinistros.append(dfaux)
    
df1 = pd.concat(listSinistros, axis = 0, ignore_index = True)

#listSinistros

b'Skipping line 223: expected 16 fields, saw 17\n'
b'Skipping line 87: expected 16 fields, saw 17\n'
b'Skipping line 27: expected 16 fields, saw 17\n'
b'Skipping line 34: expected 16 fields, saw 17\nSkipping line 38: expected 16 fields, saw 17\n'
b'Skipping line 23: expected 16 fields, saw 17\nSkipping line 72: expected 16 fields, saw 17\nSkipping line 100: expected 16 fields, saw 17\n'


In [20]:
# remove variaveis nao utilizadas
df1 = df1.drop(['cdprocesso','nrprocesso','mercadoria','coordenadas'], axis=1)

# remove registros que sejam AVARIA
df1=df1[df1['tpsinistro']!='AVARIA']

# trata tipos de dados
df1['dthrevento'] = pd.to_datetime(df1['dtevento'] + ' ' + df1['hrevento'],
                                   format='%d/%m/%Y %H:%M')
df1['dtevento'] =  pd.to_datetime(df1['dtevento'], format='%d/%m/%Y')

# cria variaveis adicionais
df1['diasemana'] = df1['dtevento'].apply( lambda x: x.dayofweek) # 0 - Segunda até 6 - Domingo
df1['nrdiasiniciomes'] = df1['dtevento'].apply( lambda x: x.day) # nr dias desde o inicio do mês

#df1.loc[df1['mercadoria'].str.contains('DIVERSOS', 
#                                       flags=re.IGNORECASE, 
#                                       regex=True)==True, 
#        'mercadoria'] = 'Diversos'

In [21]:
# obtem a data e hora do evento mais recente
df2 = df1.drop([
               'tpsinistro','tpoperacao','txtunknow','vlrgasto','placa',
               'modeloveiculo','tpveiculo','idadeveiculo',
               'naturezasinistro','dtevento','hrevento','diasemana',
                'nrdiasiniciomes'], axis=1)

df2 = df2.groupby(['nrviagem'], as_index=False).max()

# obtem o valor total gasto por viagem
df3 = df1.drop([
               'tpsinistro','tpoperacao','txtunknow','placa',
               'modeloveiculo','tpveiculo','idadeveiculo',
               'naturezasinistro','dtevento','hrevento','dthrevento',
                'diasemana','nrdiasiniciomes'], axis=1)

df3 = df3.groupby(['nrviagem'], as_index=False).sum()

df1 = df1.drop(['vlrgasto'], axis=1)

df1 = pd.merge(df1, df2, on=['nrviagem','dthrevento'], how='inner')
df1 = pd.merge(df1, df3, on=['nrviagem'], how='inner')

df1['turnoevento']=df1['dthrevento'].dt.hour
df1['turnoevento']=df1['turnoevento'].apply(set_shift)

# remove registros duplicados
df1.drop_duplicates(keep='first', inplace=True)

#df2 = df2.drop(['dthrevento'], axis=1)

del(df2)
del(df3)

In [22]:
# unifica dados finais
df = pd.merge(df, df1, on=['nrviagem'], how='left')

del(df1)

dfx=df

In [23]:
# ordena colunas no dataframe
df = df[['nrviagem','vlrtotal','urbana','cto','possuisinistro','idade','tempowf',
        't__ATC','t__LOC','t__OMN','t__ONX','t__OUTRAS','t__SAS','t__SGH','t__TRC',
        'p__1','p__2','p__3','p__4','p__5','p__6','p__7','p__9','p__10','p__11','p__12','p__13',
        'p__15','p__18','p__19','p__20','p__95','p__96','p__97','p__98','p__99',
        'c__ANAPOLIS','c__APARECIDA DE GOIANIA','c__ARAUCARIA','c__BARRA MANSA','c__BARUERI',
        'c__BAURU','c__BELEM','c__BELO HORIZONTE','c__BETIM','c__BLUMENAU','c__BRASILIA',
        'c__CAMPINAS','c__CANOAS','c__CARAZINHO','c__CAXIAS DO SUL','c__CONTAGEM','c__CUBATAO',
        'c__CUIABA','c__CURITIBA','c__DUQUE DE CAXIAS','c__EMBU DAS ARTES','c__FEIRA DE SANTANA',
        'c__FORTALEZA','c__GASPAR','c__GOIANIA','c__GUARULHOS','c__HORTOLANDIA','c__ITAJAI',
        'c__JABOATAO DOS GUARARAPES','c__JOINVILLE','c__JUNDIAI','c__LONDRINA','c__LOUVEIRA',
        'c__MANAUS','c__NAVEGANTES','c__NOVA SANTA RITA','c__OSASCO','c__OUTRAS','c__PARANAGUA',
        'c__PAULINIA','c__PORTO ALEGRE','c__RIBEIRAO PRETO','c__RIO DE JANEIRO','c__SALVADOR',
        'c__SAO BERNARDO DO CAMPO','c__SAO JOSE DOS PINHAIS','c__SAO PAULO','c__SERRA',
        'c__SIMOES FILHO','c__SUMARE','c__UBERLANDIA','c__VIANA','c__VILA VELHA',
        'c__VITORIA DE SANTO ANTAO',
        'm__ACUCAR','m__AUTO-PECAS','m__BEBIDAS','m__CALCADOS','m__CARGA FRACIONADA',
        'm__CIGARROS','m__COMBUSTIVEL','m__COSMETICOS','m__DEFENSIVO AGRICOLA',
        'm__DIVERSOS','m__DIVERSOS PRODUTOS','m__EQUIPAMENTOS ELETRICOS','m__ESPECIFICOS',
        'm__ESTIRENO','m__FIOS E CABOS','m__MEDICAMENTOS','m__NESTLE','m__OUTRAS',
        'm__PECAS AUTOMOTIVAS','m__POLIESTIRENO','m__POLIETILENO','m__PRODUTO ACABADO',
        'm__PRODUTOS QUIMICOS','m__TINTAS E SOLVENTES',
        'tpsinistro','nrdiasiniciomeswf','diasemanawf',
        'salmoco','sjanta','smadrugada','smanha','snoite','starde']]

#pd.set_option('display.max_columns', 500)
df['vlrtotal'] = pd.to_numeric(df['vlrtotal'].str.replace(',', '.'))

#df['vlrtotal'] = pd.to_numeric(df['vlrtotal'])
df['urbana'] = df['urbana'].astype('int64')

#df.head()
#df['urbana']

In [24]:
# gera dados de WF
df.to_csv('/home/dev/poc/arquivos/model/out/multisat-v6.csv',index=False)

In [25]:
# gera dados de roubo
roubo=df[df['tpsinistro']!='ACIDENTE']
roubo.loc[roubo['tpsinistro']=='ROUBO','tpsinistro']=1
roubo.to_csv('/home/dev/poc/arquivos/model/out/roubo-v6.csv',index=False)

In [26]:
# gera dados de acidente
acidente=df[df['tpsinistro']!='ROUBO']
acidente.loc[acidente['tpsinistro']=='ACIDENTE','tpsinistro']=1
acidente.to_csv('/home/dev/poc/arquivos/model/out/acidente-v6.csv',index=False)